<a href="https://colab.research.google.com/github/Cath-Strategic-Tech/adpdx_etl/blob/main/ADPDX_ClergyDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The following notebook orchestrates the migration of ADPDX Accounts into Salesforce.




# Order of Operations

- Setup Enviro
    - [DONE] UDFs
    - [DONE] Load SF xref data

- ACCOUNTS
    - Extract Source Data
        - [DONE] Load 6 tables into separate dataframes
        - [DONE] Merge into single accounts table
        - #TODO Fix the ExternalID so that it references the original table, not the AccRecordType
    - Transform
    - Load
        - [DONE]Vicariates
        - [DONE] Organizations (Parishes, Schools, Newman Centres, Offices)
        - Religious
            - [DONE] Religious Parent accounts
            - [DONE] Religious Communities
            - [DONE] Religious Superiors (Contacts, set AccountID to Rel. Parent)
            - #TODO Update Religious Communities with lookup to Rel. Superior

- CONTACTS
    - Extract
        - [DONE] Import Contact records
        - #TODO Get Photo directory @soames
    
    - Analysis
        - [DONE] Check columns & row count (3016)
        - [DONE] Identify unique languages

    - Transform
        - Complete ETL of fields that are more complex (search for #TODO)
        - [DONE] Create new df_contact_staging, renaming columns to SF APIs
        - [DONE] Drop columns that don't map to Contact
        - Migrate Languages field (waiting on next package version) @soames
            - #TODO: transform `,` to `;` so imports to multi-select list correctly
        - #TODO Concat Mailing Street Address lines into one
        - #TODO Handle Private Addresses:  decide if will code changes or NOT use a custom Private Address field.
        - [DONE] Update boolean fields to True/False
        - [DONE] Set Contact Record Type (UDF)
        - [DONE] Validate, drop invalid emails
        - [DONE] Generate ExternalID > 'Archdpdx_External_Id__c'
        - #TODO Preferred Email/Phone > where blank, set a default. Currently, all are getting set to 'Personal' and 'Mobile.'
        - #TODO Ecclesial Status (not mapping correctly)
        - [DONE] DROP columns that haven't been mapped yet

    - Load
        - [DONE] Set JobID to curr_job_id
        - [DONE] Handle character encoding that is geting messed up
        - #TODO Fix why the simple-salesforce insert isn't working

- CONTACTS > SPOUSES
    - #TODO

- CONTACTS > PHOTOS
    - [DONE] Investigate how to migrate photos into a RTF (rich text field)
    - #TODO

- CONTACTS > REGISTER ENTRIES
    - Parse columns into types of Sacraments or Notations
    - For lookups to Celebrants, query SF for contacts, create missing records
    - Generate External ID, apply to df
    - Clean up (remove extra columns, NaNs)
    - Upsert records

- AFFILIATIONS

    - Extract
        - [DONE] Turn the 'Org Table Name' & 'org Table Link' columns into External ID
        - Map in the Account IDs from SF
        -

    - Transform
        - Parse RecordTypeId
        - Parse Category
        - Map columns to SF field APIs

    - Load


# Setup Enviro

In [2]:
# enviro setup

import pandas as pd

from google.colab import data_table
data_table.enable_dataframe_formatter()

from google.colab import userdata

from datetime import datetime
now = datetime.now()

try:
    from simple_salesforce import Salesforce
except:
    !pip install simple-salesforce
    from simple_salesforce import Salesforce

from IPython.display import clear_output
clear_output()

In [3]:
# Import the MFC_Functions notebook
!pip install import-ipynb
import import_ipynb
clear_output()

In [4]:
!pip install email_validator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 7.4 MB/s eta 0:00:00


In [5]:
# @title Global Variables { run: "auto", vertical-output: true, display-mode: "both" }

target_enviro = "adpdx_devpro" # @param {type:"string"}

diocesan_account_id_devpro = "001Dx00001CwMTQIA3" # @param {type:"string"}

# @markdown The `run_upserts` variable controls whether or not upserts to Salesforce are executed when the notebook is run.
run_upserts = "False" # @param ["True", "False"]

In [6]:
# ADPDX prod credentials
prod_pass = userdata.get('portland_prod_user')
prod_token = userdata.get('portland_prod_securitytiken')

# ADPDX dev_pro credentials
adpdx_devpro_pass = userdata.get('adpdx_devpro_pass')
adpdx_devpro_token = userdata.get('adpdx_devpro_token')

# instantiate a SF session object
sf = Salesforce(domain='test', username='matt+adpdx@meribahflow.com.devpro', password=adpdx_devpro_pass, security_token=adpdx_devpro_token)

## UDFs



In [7]:
# Job ID Incrementer

def update_job_id(file_name):
    # Open the file in read mode and get the current job ID
    with open(file_name, 'r') as file:
        current_job_id = int(file.readline())

    # Increment the job ID
    new_job_id = current_job_id + 1

    # Open the file in write mode and update the job ID
    with open(file_name, 'w') as file:
        file.write(str(new_job_id))

    # Return the new job ID
    return new_job_id


# Concates two DF columns for an External ID

def concat_columns(df, columns, new_column, separator='_'):
    """
    Concatenates the values from specified columns into a single string
    with the specified separator and populates a new column in the DataFrame.

    Args:
    - df: pandas DataFrame
    - columns: list of column names to concatenate
    - new_column: name of the new column to be created
    - separator: separator to use between concatenated values (default is '_')

    Returns:
    - Updated pandas DataFrame with the new column
    """
    df[new_column] = df[columns].astype(str).apply(lambda x: separator.join(x), axis=1)
    return df



## Extract Salesforce xref data
The following cells downloads all records from the target Salesforce enviro for the following objects:

* RecordTypes
* Users
* Accounts
* Contacts

In [8]:
# get all ACTIVE SF users

sf_users = sf.query('Select Alias, FirstName, LastName, Username, id from User WHERE IsActive = True')
df_sf_users = pd.DataFrame(sf_users['records'])
df_sf_users = df_sf_users.drop(columns = 'attributes')
df_sf_users.shape

(8, 5)

In [9]:
# get all SF Record Types
get_all_recordTypes = 'Select Id, Name, DeveloperName, sObjecttype, namespaceprefix from RecordType'

# get list of records, add to dataframe
sf_recordTypes = sf.query(get_all_recordTypes)
df_sf_recordTypes = pd.DataFrame(sf_recordTypes['records'])
df_sf_recordTypes = df_sf_recordTypes.drop(columns = 'attributes')

# Create a dictionary mapping 'DeveloperName' to 'Id' for faster lookup
record_types_mapping = df_sf_recordTypes.set_index('DeveloperName')['Id'].to_dict()

df_sf_recordTypes

,Id,Name,DeveloperName,SobjectType,NamespacePrefix
0,012Dx0000007yCpIAI,Property,Property,Account,None
1,012Dx0000007yIOIAY,Ecclesial Affiliation,Ecclesial_Affiliation,mbfc__Placement__c,None
2,012Dx0000007yITIAY,Pastoral Assignments,Assignments_Clergy,mbfc__Placement__c,None
3,012Dx0000007yIYIAY,Staff,Staff,mbfc__Placement__c,None
4,012Dx0000007yIdIAI,Lay Person,Lay_Person,mbfc__Placement__c,None
5,012Dx0000007yOCIAY,Diocesan Appointment,Diocesan_Appointment,mbfc__Placement__c,None
6,012Dx0000007yOHIAY,Clergy/Religious Residence,Clergy_Religious_Residence,mbfc__Placement__c,None
7,012Dx0000007yOMIAY,Education,Education,mbfc__Placement__c,None
8,012Dx0000007yORIAY,Ministerial Status,Ministerial_Status,mbfc__Placement__c,None
9,012Dx0000007yTgIAI,z) All Types,All_Types,mbfc__Placement__c,None


In [10]:
# get SF Account
get_all_accounts = 'Select id, Name, RecordTypeId, Type, mbfc__Parish_Code__c, Job_Id__c, Archdpdx_Migration_Id__c from Account'

# get list of records, add to dataframe
sf_accounts = sf.query(get_all_accounts)
df_sf_accounts = pd.DataFrame(sf_accounts['records'])
df_sf_accounts = df_sf_accounts.drop(columns = 'attributes')
df_sf_accounts.shape

(2000, 7)

In [11]:
# get SF Contacts
get_all_contacts = 'Select id, Name, npe01__Type_of_Account__c, RecordTypeId, CreatedById from Contact'

# get list of records, add to dataframe
sf_contacts = sf.query(get_all_contacts)
df_sf_contacts = pd.DataFrame(sf_contacts['records'])
df_sf_contacts = df_sf_contacts.drop(columns = 'attributes')
df_sf_contacts.shape

(2000, 5)

# ACCOUNTS

## Extract

### Load ArchdPDX csvs as DataFrames
ADPDX data for organizations is held in 6 tables, all of which will be migrated into Salesforce's Accounts object.  

In [12]:
df_offices = pd.read_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Offices.csv', skiprows= lambda x: x in [1])
df_offices["src_table"] = 'Offices'
df_offices["AccountRecordType"] = 'Organization'
df_offices.rename({"Name": "Account Name"}, axis="columns", inplace=True)
df_offices.columns

Index(['Record Number', 'Common Name', 'Account Name',
       'Archdiocese Assigns Clergy', 'Locator Description', 'Mailing Address',
       'Mailing Address 2', 'Mailing Address City', 'Mailing Address State',
       'Mailing Address Province', 'Mailing Address Postal Code',
       'Mailing Address Country', 'Phone', 'Fax', 'Email', 'Web Site',
       'src_table', 'AccountRecordType'],
      dtype='object')

In [13]:
df_parishes = pd.read_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Parishes (3).csv', dtype={'Vicariate': 'object'}, skiprows= lambda x: x in [1])
df_parishes["src_table"] = 'Parishes'
df_parishes["AccountRecordType"] = 'Church'
df_parishes.rename({"Parish Formal Name": "Account Name"}, axis="columns", inplace=True)
df_parishes.columns

df_parishes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 45 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Record Number                 151 non-null    int64  
 1   Common Name                   151 non-null    object 
 2   Sort Name                     151 non-null    object 
 3   Parish Name                   5 non-null      object 
 4   Account Name                  151 non-null    object 
 5   Parish City                   151 non-null    object 
 6   Archdiocese Assigns Clergy    151 non-null    object 
 7   Mission Of                    151 non-null    int64  
 8   Established                   151 non-null    int64  
 9   Vicariate                     151 non-null    object 
 10  Non-Latin                     151 non-null    object 
 11  Locator Description           89 non-null     object 
 12  Mailing Address               151 non-null    object 
 13  Maili

In [14]:
df_religious = pd.read_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/RelCommunities.csv', skiprows= lambda x: x in [1])
df_religious["src_table"] = 'RelCommunities'
df_religious["AccountRecordType"] = 'Religious'
df_religious.rename({"Community Name": "Account Name"}, axis="columns", inplace=True)
df_religious.columns

Index(['Record Number', 'Common Name', 'Account Name', 'Community City',
       'Archdiocese Assigns Clergy', 'Order Full Name', 'Order Common Name',
       'Order Letters', 'Men or Women', 'Non-Latin Rite', 'Show Order in Name',
       'Description', 'Locator Description', 'Mailing Address',
       'Mailing Address 2', 'Mailing Address City', 'Mailing Address State',
       'Mailing Address Province', 'Mailing Address Postal Code',
       'Mailing Address Country', 'Phone', 'Fax', 'Email', 'Web Site',
       'Religious Order', 'Secular Order', 'Diocesan Order',
       'Pontifical Order', 'Local Superior', 'Major Superior Name',
       'Major Superior Phone', 'Major Superior Email', 'src_table',
       'AccountRecordType'],
      dtype='object')

In [15]:
df_schools = pd.read_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Schools.csv', skiprows= lambda x: x in [1])
df_schools["src_table"] = 'Schools'
df_schools["AccountRecordType"] = 'School'
df_schools.rename({"School Name": "Account Name"}, axis="columns", inplace=True)
df_schools.columns

Index(['Record Number', 'Common Name', 'Account Name', 'School City',
       'Archdiocese Assigns Clergy', 'Parish Link', 'Vicariate Link',
       'Archdiocesan School Code', 'Grades Provided', 'Established',
       'Locator Description', 'Mailing Address 1', 'Mailing Address 2',
       'Mailing Address City', 'Mailing Address State',
       'Mailing Address Province', 'Mailing Address Zip',
       'Mailing Address Country', 'Phone', 'Fax', 'Email', 'Web Site',
       'Lat/Long Coordinates Decimal', 'Google Small Embed URL', 'src_table',
       'AccountRecordType'],
      dtype='object')

In [16]:
df_vicariates = pd.read_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Vicariates.csv', skiprows= lambda x: x in [1])
df_vicariates["src_table"] = 'Vicariates'
df_vicariates["AccountRecordType"] = 'Deanery'
# As we want to designate the Common Name as what will be the Account Name in Salesforce, we are renaming these columns in a different pattern than prior CSVs.
df_vicariates.rename({"Common Name": "Account Name"}, axis="columns", inplace=True)

df_vicariates.columns

Index(['Record Number', 'Account Name', 'Vicariate Name',
       'Archdiocese Assigns Clergy', 'src_table', 'AccountRecordType'],
      dtype='object')

In [17]:
df_newman = pd.read_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/NewmanCenters.csv', skiprows= lambda x: x in [1])
df_newman["src_table"] = 'NewmanCenters'
df_newman["AccountRecordType"] = 'Organization'
df_newman.rename({"Newman Center Name": "Account Name", "Newman Center City": "Mailing Address City2"}, axis="columns", inplace=True)
df_newman.shape

(4, 37)

Each of the 6 tables has an overlapping but distinct set of columns, making it challenging to conform these tables into a single staging table.  

In addition, columns that correspond to the same field in salesforce are named differently in each table (eg. 'Parish City' vs. 'Religious City' vs. 'Newman Center City')

In [18]:
print('TABLE: (ROWS, COLUMNS)\n')

print(f'Offices:    {df_offices.shape}')
print(f'Parishes:   {df_parishes.shape}')
print(f'Religious:  {df_religious.shape}')
print(f'Schools:    {df_schools.shape}')
print(f'Vicariates: {df_vicariates.shape}')
print(f'Newman Ctr: {df_newman.shape}')

TABLE: (ROWS, COLUMNS)

Offices:    (35, 18)
Parishes:   (151, 45)
Religious:  (70, 34)
Schools:    (56, 26)
Vicariates: (18, 6)
Newman Ctr: (4, 37)


### Merge DFs into a single Accounts DF
This step takes 6 different tables and combines them into a single Accounts table for cleaning and staging.

In [19]:
# init list of DataFrames
src_accounts = [df_offices, df_parishes, df_religious, df_schools, df_vicariates, df_newman]

# concats the various Account dataframes into one large table
accounts = pd.concat(src_accounts, ignore_index=True)

accounts.head(5)

,Record Number,Common Name,Account Name,Archdiocese Assigns Clergy,Locator Description,Mailing Address,Mailing Address 2,Mailing Address City,Mailing Address State,Mailing Address Province,...,Major Superior Email,School City,Parish Link,Vicariate Link,Archdiocesan School Code,Grades Provided,Mailing Address 1,Mailing Address Zip,Vicariate Name,Mailing Address City2
0,1,Pastoral Center,Pastoral Center,Yes,NaN,2838 E Burnside St,NaN,Portland,OR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,Catholic Sentinel,Catholic Sentinel,No,NaN,2838 E Burnside St,NaN,Portland,OR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,Catholic Cemeteries,Catholic Cemeteries,No,NaN,333 SW Skyline Blvd,NaN,Portland,OR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,Griffin Center,Griffin Center,No,NaN,11957 SE Fuller Rd,NaN,Milwaukie,OR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,Providence Portland Medical Center,Providence Portland Medical Center,Yes,NaN,4805 NE Glisan St,NaN,Portland,OR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Time to do some table column renaming and re-organizing!

In [20]:
# renames columns headers to consolidate account names into SF-conformed data model
accounts.rename({"Common Name": "Name, City"}, axis="columns", inplace=True)

accounts.rename(
    columns={
        'Account Name': 'Name',
        'Mailing Address': 'BillingStreet',
        'Mailing Address 2': 'BillingStreet2',
        'Mailing Address City': 'BillingCity',
        'Mailing Address State': 'BillingState',
        'Mailing Address Postal Code': 'BillingPostalCode',
        'Mailing Address Country': 'BillingCountry',
        'Email': 'mbfc__Email__c',
        'Web Site': 'Website',
        'Order Common Name': 'mbfc__Abbreviation__c',
        'Order Letters': 'mbfc__Religious_Suffix__c',
        'Men or Women': 'mbfc__Type_Members__c'
    },
    inplace=True
)

# reorder column order
col = accounts.pop('Name')
accounts.insert(2, col.name, col)

col = accounts.pop('Parish Name')
accounts.insert(3, col.name, col)

col = accounts.pop('AccountRecordType')
accounts.insert(1, col.name, col)

accounts[accounts.BillingStreet2.isna() == False]

,Record Number,AccountRecordType,"Name, City",Name,Parish Name,Archdiocese Assigns Clergy,Locator Description,BillingStreet,BillingStreet2,BillingCity,...,Major Superior Email,School City,Parish Link,Vicariate Link,Archdiocesan School Code,Grades Provided,Mailing Address 1,Mailing Address Zip,Vicariate Name,Mailing Address City2
14,32,Organization,Diaconate Office,Diaconate Office,NaN,Yes,NaN,Pastoral Center,2838 E Burnside St,Portland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,58,Organization,Office of Marketing and Communications,Office of Marketing and Communications,NaN,Yes,NaN,Pastoral Center,2838 E Burnside St,Portland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,1,Church,"Our Lady of Perpetual Help, St Mary’s, Albany","Our Lady of Perpetual Help, St Mary’s",NaN,Yes,SW Ellsworth St between 8th and 9th Streets,"Our Lady of Perpetual Help, St Mary’s Parish",815 Broadalbin St SW,Albany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,2,Church,"St. Andrew Dũng-Lạc Mission, Aloha",St. Andrew Dũng-Lạc,NaN,No,SW Grabhorn Rd/209th Ave and Farmington Rd,St. Andrew Dũng-Lạc Mission,7390 SW Grabhorn Rd,Aloha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,3,Church,"St. Elizabeth Ann Seton, Aloha",St. Elizabeth Ann Seton,NaN,Yes,NaN,St. Elizabeth Ann Seton Parish,3145 SW 192nd Ave,Aloha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,62,Religious,"Work of Jesus the High Priest, Gresham (OJSS)",Work of Jesus the High Priest,NaN,No,NaN,OJSS Community,451 NW 1st St,Gresham,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,64,Religious,"Heralds of the Good News, Portland (HGN)",Heralds of the Good News,NaN,No,NaN,c/o Chancellor,2838 E Burnside St,Portland,...,rkappumkal@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239,65,Religious,"Missionary Oblates of Mary Immaculate, Rome, I...",Missionary Oblates of Mary Immaculate,NaN,No,NaN,Missionary Oblates of Mary Immaculate,Via Aurelia 290,Roma,...,gensec@omigen.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,73,Religious,"Brothers of Saint John, Laredo, TX (CSJ)",Brothers of Saint John,NaN,No,NaN,St. John Priory,505 Century Dr S,Laredo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# export merged tables DESCRIPTION to CSV for mapping
accounts.describe(include='all').transpose().to_csv(f'/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/working/accounts.csv')
accounts.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Record Number,334.0,NaN,NaN,NaN,54.5,41.389801,1.0,21.25,45.0,76.75,173.0
AccountRecordType,334,5,Church,151,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Name, City",316,316,Pastoral Center,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Name,334,291,St. Mary,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Parish Name,5,5,St. Anne,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
Grades Provided,52,12,PS-8,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mailing Address 1,56,55,4420 SW St Marys Dr,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mailing Address Zip,56.0,NaN,NaN,NaN,97222.446429,124.9586,97005.0,97134.75,97217.5,97301.0,97526.0
Vicariate Name,18,18,Albany-Corvallis,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Transform

TODO:
- Strip phone numbers
- Validate email addresses
- Concat BillingStreet 1 + BillingStreet 2 into single field
- For all Accounts with 'Primary Contacts' or 'Major Superiors', add Contact to staging DF, to be created after parent account is created.
- Handle churches that aren't parishes (missions, non-diocesan parishes, etc.)

In [22]:
# Create a single BillingAddress field
# billingstreet = str(f"Mailing Addresss 1 /n Mailing Address 2")
# accounts['BillingStreet'] = accounts['Mailing Address 1'].astype(str) + accounts['Mailing Address 2'].astype(str)


### AccountRecordType & ChurchType

In [23]:
# Sets all rows where AccountRecordType is Church as a Parish. THIS MAY NEED NUANCING
accounts.loc[accounts['AccountRecordType'] == 'Church', 'mbfc__Church_Type__c'] = 'Parish'
accounts[accounts['AccountRecordType'] == 'Church'].head(5)


,Record Number,AccountRecordType,"Name, City",Name,Parish Name,Archdiocese Assigns Clergy,Locator Description,BillingStreet,BillingStreet2,BillingCity,...,School City,Parish Link,Vicariate Link,Archdiocesan School Code,Grades Provided,Mailing Address 1,Mailing Address Zip,Vicariate Name,Mailing Address City2,mbfc__Church_Type__c
35,1,Church,"Our Lady of Perpetual Help, St Mary’s, Albany","Our Lady of Perpetual Help, St Mary’s",NaN,Yes,SW Ellsworth St between 8th and 9th Streets,"Our Lady of Perpetual Help, St Mary’s Parish",815 Broadalbin St SW,Albany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish
36,2,Church,"St. Andrew Dũng-Lạc Mission, Aloha",St. Andrew Dũng-Lạc,NaN,No,SW Grabhorn Rd/209th Ave and Farmington Rd,St. Andrew Dũng-Lạc Mission,7390 SW Grabhorn Rd,Aloha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish
37,3,Church,"St. Elizabeth Ann Seton, Aloha",St. Elizabeth Ann Seton,NaN,Yes,NaN,St. Elizabeth Ann Seton Parish,3145 SW 192nd Ave,Aloha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish
38,4,Church,"St. Peter the Fisherman Mission, Arch Cape",St. Peter the Fisherman,NaN,Yes,79441 Hwy 101 S,St. Peter the Fisherman Mission,PO Box 29,Seaside,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish
39,5,Church,"Our Lady of the Mountain, Ashland",Our Lady of the Mountain,NaN,Yes,NaN,Our Lady of the Mountain Parish,987 Hillview Dr,Ashland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish


In [24]:
# Generate an External ID
columns_to_concate = ['AccountRecordType', 'Record Number']
accounts = concat_columns(accounts, columns_to_concate, 'Archdpdx_Migration_Id__c', separator='_')

In [25]:
# set Deanery RecordTypeId to the Church RecordTypeId
# map in RecordTypeIds
accounts['RecordTypeId'] = accounts['AccountRecordType'].map(record_types_mapping)
record_types_mapping

{'Property': '012Dx0000007yCpIAI',
 'Ecclesial_Affiliation': '012Dx0000007yIOIAY',
 'Assignments_Clergy': '012Dx0000007yITIAY',
 'Staff': '012Dx0000007yIYIAY',
 'Lay_Person': '012Dx0000009TK3IAM',
 'Diocesan_Appointment': '012Dx0000007yOCIAY',
 'Clergy_Religious_Residence': '012Dx0000007yOHIAY',
 'Education': '012Dx0000009TKBIA2',
 'Ministerial_Status': '012Dx0000007yORIAY',
 'All_Types': '012Dx0000007yTgIAI',
 'Church': '012Dx0000009TJxIAM',
 'Deanery': '012Dx0000009TJyIAM',
 'Group': '012Dx0000009TJzIAM',
 'Religious': '012Dx0000009TK6IAM',
 'School': '012Dx0000009TK1IAM',
 'Consecrated': '012Dx0000009TK2IAM',
 'Permanent_Deacon': '012Dx0000009TK4IAM',
 'Priest': '012Dx0000009TK5IAM',
 'All': '012Dx0000009TK7IAM',
 'Chancery_Users': '012Dx0000009TK8IAM',
 'Diocean_Users': '012Dx0000009TK9IAM',
 'Parish_Users': '012Dx0000009TKAIA2',
 'Employment': '012Dx0000009TKCIA2',
 'Ministry_Volunteer': '012Dx0000009TKDIA2',
 'Parishioner': '012Dx0000009TKEIA2',
 'MajorGift': '012Hu000001pkqBIAQ'

In [26]:
#TODO: Set 'organization type' field value for each account in the 'organization' load: Offices, Newman Centres, Schools, Organizations
# Might be best if it is set manually at the end of the migration.

## Load


### Generate a new Job ID



In [27]:
# increment to the job_id
file_name = '/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/jobs/job_id'
curr_job_id = update_job_id(file_name)
print(f"New job ID: {curr_job_id}")

# add/update account DF with job_id
accounts["Job_Id__c"] = curr_job_id


New job ID: 45


### A) Vicariates

In [28]:
vicariates = accounts[accounts['AccountRecordType'] == 'Deanery']

vicariates = vicariates[[
    'Record Number',
    'Name',
    # 'AccountRecordType',
    'Job_Id__c',
    'Archdpdx_Migration_Id__c',
    'RecordTypeId'
    ]]

# add parentid
vicariates["mbfc__Diocese__c"] = diocesan_account_id_devpro
vicariates['ParentId'] = diocesan_account_id_devpro
vicariates['mbfc__Church_Type__c'] = 'Deanery'

vicariates.rename(columns={
        # 'Name, City': 'Name',
        'External_Id': 'Archdpdx_Migration_Id__c'
    }, inplace=True)

vicariates.reset_index()
vicariates.set_index('Record Number', inplace=True)

vicariates

,Name,Job_Id__c,Archdpdx_Migration_Id__c,RecordTypeId,mbfc__Diocese__c,ParentId,mbfc__Church_Type__c
Record Number,,,,,,,
1,Albany-Corvallis Vicariate,45,Deanery_1,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery
2,"Beaverton, Suburban Vicariate",45,Deanery_2,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery
3,Columbia County Vicariate,45,Deanery_3,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery
4,Downtown Portland Vicariate,45,Deanery_4,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery
5,"East Portland, Suburban Vicariate",45,Deanery_5,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery
6,Marion County Vicariate,45,Deanery_6,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery
7,Metropolitan Eugene Vicariate,45,Deanery_7,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery
8,Metropolitan Salem Vicariate,45,Deanery_8,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery
9,North Coast Vicariate,45,Deanery_9,012Dx0000009TJyIAM,001Dx00001CwMTQIA3,001Dx00001CwMTQIA3,Deanery


#### Export Vicariates to CSV

In [29]:
# export to CSV
vicariates.to_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/vicariates_staging.csv')

#### Upsert Vicariates

In [30]:
bulk_data = []
for row in vicariates.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)

if run_upserts == 'True':
    vicariate_upsert = sf.bulk.Account.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
    upserts = pd.DataFrame(vicariate_upsert)

In [31]:
# create a map of Vicariate lookup ids to the unique ids generated on the Vicariate records
# vicariates_externalid_map = vicariates.set_index('Record Number')['Archdpdx_Migration_Id__c'].to_dict()

# vicariates_externalid_map

In [32]:
# Generate an Errors log
import csv

keys = vicariate_upsert[0].keys()

with open('vicariate_results', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, keys)
    writer.writeheader()
    writer.writerows(vicariate_upsert)

In [33]:
# @title Get Vicariate records from SF

sf_deaneries = sf.query("SELECT Archdpdx_Migration_Id__c, Id FROM Account WHERE RecordType.DeveloperName = 'Deanery'")

df_sf_deaneries = pd.DataFrame(sf_deaneries['records'])
df_sf_deaneries = df_sf_deaneries.drop(columns = 'attributes')

df_sf_deaneries

# Creates a dict of Vicariate unique ids to the new Salesforce record IDs, so can populate on latter Account records
vicariate_sf_recordids = df_sf_deaneries.set_index('Archdpdx_Migration_Id__c')['Id'].to_dict()
vicariate_sf_recordids

{'Deanery_1': '001Dx00001CwdOmIAJ',
 'Deanery_2': '001Dx00001CwdOnIAJ',
 'Deanery_3': '001Dx00001CwdOoIAJ',
 'Deanery_4': '001Dx00001CwdOpIAJ',
 'Deanery_5': '001Dx00001CwdOqIAJ',
 'Deanery_6': '001Dx00001CwdOrIAJ',
 'Deanery_7': '001Dx00001CwdOsIAJ',
 'Deanery_8': '001Dx00001CwdOtIAJ',
 'Deanery_9': '001Dx00001CwdOuIAJ',
 'Deanery_10': '001Dx00001CwdOvIAJ',
 'Deanery_11': '001Dx00001CwdOwIAJ',
 'Deanery_12': '001Dx00001CwdOxIAJ',
 'Deanery_13': '001Dx00001CwdOyIAJ',
 'Deanery_14': '001Dx00001CwdOzIAJ',
 'Deanery_15': '001Dx00001CwdP0IAJ',
 'Deanery_16': '001Dx00001CwdP1IAJ',
 'Deanery_17': '001Dx00001CwdP2IAJ',
 'Deanery_18': '001Dx00001CwdP3IAJ'}

### B) Parishes, Schools, Organizations

In [34]:
# Create a new DF with Account records - excluding Deaneries (already handled) and Religious (to be handled differently, after)
acc_main = accounts[accounts['AccountRecordType'] != 'Deanery']
acc_main = acc_main[acc_main['AccountRecordType'] != 'Religious']

acc_main.loc[acc_main['AccountRecordType'] == 'Church', 'Vicariate_Ext_Id'] = 'Deanery_' + acc_main['Vicariate']

acc_main.sample(5)

,Record Number,AccountRecordType,"Name, City",Name,Parish Name,Archdiocese Assigns Clergy,Locator Description,BillingStreet,BillingStreet2,BillingCity,...,Grades Provided,Mailing Address 1,Mailing Address Zip,Vicariate Name,Mailing Address City2,mbfc__Church_Type__c,Archdpdx_Migration_Id__c,RecordTypeId,Job_Id__c,Vicariate_Ext_Id
171,142,Church,"St. Boniface, Sublimity",St. Boniface,NaN,Yes,NaN,St. Boniface Parish,375 SE Church St,Sublimity,...,NaN,NaN,NaN,NaN,NaN,Parish,Church_142,012Dx0000009TJxIAM,45,Deanery_11
4,11,Organization,Providence Portland Medical Center,Providence Portland Medical Center,NaN,Yes,NaN,4805 NE Glisan St,NaN,Portland,...,NaN,NaN,NaN,NaN,NaN,NaN,Organization_11,012Hu000001pkqEIAQ,45,NaN
135,106,Church,"St. Philip Neri, Portland",St. Philip Neri,NaN,Yes,SE 16th Ave and Division St,St. Philip Neri Parish,2408 SE 16th Ave,Portland,...,NaN,NaN,NaN,NaN,NaN,Parish,Church_106,012Dx0000009TJxIAM,45,Deanery_14
43,11,Church,"St. Cecilia, Beaverton",St. Cecilia,NaN,Yes,NaN,St. Cecilia Parish,5105 SW Franklin Ave,Beaverton,...,NaN,NaN,NaN,NaN,NaN,Parish,Church_11,012Dx0000009TJxIAM,45,Deanery_2
60,29,Church,"St. Aloysius, Estacada",St. Aloysius,NaN,Yes,192 NW 3rd Ave,St. Aloysius Parish,18090 SE Langensand Rd,Sandy,...,NaN,NaN,NaN,NaN,NaN,Parish,Church_29,012Dx0000009TJxIAM,45,Deanery_13


In [35]:
# TODO: This is now obsolete (I think)
# acc_main['VicariateUniqueId'] = acc_main['Vicariate'].map(vicariates_externalid_map)

acc_main['mbfc__Deanery__c'] = acc_main.Vicariate_Ext_Id.map(vicariate_sf_recordids)

acc_main[acc_main['AccountRecordType'] == 'Church']['mbfc__Deanery__c']

35     001Dx00001CwdOmIAJ
36     001Dx00001CwdOyIAJ
37     001Dx00001CwdP1IAJ
38     001Dx00001CwdOuIAJ
39     001Dx00001CwdP0IAJ
              ...        
181    001Dx00001CwdOqIAJ
182    001Dx00001CwdP2IAJ
183    001Dx00001CwdOrIAJ
184    001Dx00001CwdP3IAJ
185    001Dx00001CwdOsIAJ
Name: mbfc__Deanery__c, Length: 151, dtype: object

In [36]:
# Clean up NaN values

acc_main.fillna('', inplace=True)

In [37]:
# @title Export to CSV
# Export to CSV for manual loading

accounts_staging = acc_main[[
    'Name',
    'RecordTypeId',
    'mbfc__Church_Type__c',
    'mbfc__Deanery__c',
    'BillingStreet',
    'BillingCity',
    'BillingState',
    'BillingPostalCode',
    'BillingCountry',
    'Phone',
    'Fax',
    'mbfc__Email__c',
    'Website',
    'mbfc__Abbreviation__c',
    'mbfc__Religious_Suffix__c',
    'mbfc__Type_Members__c',
    'Description',
    'Job_Id__c',
    'Archdpdx_Migration_Id__c'

    ]]

In [38]:
accounts_staging.to_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/accounts_staging.csv', encoding='utf-8-sig')

#### Upsert Accounts (TBD    )

In [39]:
bulk_data = []
for row in accounts_staging.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)

In [40]:
if run_upserts == 'True':
    account_staging_upsert = sf.bulk.Account.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
    account_upserts = pd.DataFrame(account_staging_upsert)

account_upserts

,success,created,id,errors
0,True,False,001Dx00001CwhxOIAR,[]
1,True,False,001Dx00001CwhxPIAR,[]
2,True,False,001Dx00001CwhxQIAR,[]
3,True,False,001Dx00001CwhxRIAR,[]
4,True,False,001Dx00001CwhxSIAR,[]
...,...,...,...,...
241,True,False,001Dx00001CwhyqIAB,[]
242,True,False,001Dx00001CwhxOIAR,[]
243,True,False,001Dx00001CwhyrIAB,[]
244,True,False,001Dx00001CwhxPIAR,[]


In [41]:
# Generate an Errors log
import csv

keys = account_staging_upsert[0].keys()

with open('accounts_results', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, keys)
    writer.writeheader()
    writer.writerows(account_staging_upsert)

# TODO: Convert this into a UDF

In [42]:
# @title Extract SF Account records

sf_accounts = sf.query('Select id, Name, RecordTypeId, mbfc__Church_Type__c, Archdpdx_Migration_Id__c, Job_Id__c from Account WHERE Job_Id__c != null')
sf_accounts = pd.DataFrame(sf_accounts['records'])
sf_accounts = sf_accounts.drop(columns = 'attributes')
sf_accounts

,Id,Name,RecordTypeId,mbfc__Church_Type__c,Archdpdx_Migration_Id__c,Job_Id__c
0,001Dx00001DOFt5IAH,"Our Lady of Perpetual Help Church, St Mary's, ...",012Dx0000009TJxIAM,Church Building,Church_1_cloned_by_sl,12
1,001Dx00001FZm6kIAD,St. Francis of Assisi School,012Hu000001pkqEIAQ,None,Organization_5,36
2,001Dx00001FZm6lIAD,Valley Catholic Elementary School,012Hu000001pkqEIAQ,None,Organization_8,36
3,001Dx00001FZm6mIAD,Valley Catholic Middle School,012Hu000001pkqEIAQ,None,Organization_9,36
4,001Dx00001FZm6nIAD,Valley Catholic High School,012Hu000001pkqEIAQ,None,Organization_10,36
...,...,...,...,...,...,...
349,001Dx00001CwhyoIAB,University of Portland,012Dx0000009TK1IAM,None,School_56,45
350,001Dx00001CwhypIAB,Mount Angel Seminary,012Dx0000009TK1IAM,None,School_57,45
351,001Dx00001CwhyqIAB,Resurrection Catholic Parish School,012Dx0000009TK1IAM,None,School_58,45
352,001Dx00001CwhyrIAB,St. Thomas More (UO) Newman Center,012Hu000001pkqEIAQ,None,Organization_2,45


### C) Religious Institutes





In [43]:
"""
- 'acc_religious' DF: create unique_id of religious parents
- create 'acc_religious_orders' DF , upsert into SF
- extract accounts from Salesforce, create dict (external_ID : account_ID)
- map parent ids onto religious child accounts DF in main DF
- 'acc_religious' > staging DF ('acc_religious')
    - drop unnecessary columns
    - upsert create DF of religious children, upsert into SF with
"""

# Create a new DF of all Religious accounts
acc_religious = accounts[accounts['AccountRecordType'] == 'Religious']

# Create a simplified external ID field
acc_religious['Archdpdx_Migration_Id__c'] = acc_religious['Order Full Name'].apply(
    lambda x: x.lower().replace(' ', '')[:40]
)

acc_religious_2 = acc_religious

# Create a DF for only parent religious order accounts
acc_religious_parents = acc_religious_2[['Order Full Name', 'Name', 'mbfc__Abbreviation__c', 'mbfc__Religious_Suffix__c', 'mbfc__Type_Members__c', 'Archdpdx_Migration_Id__c']]

# Drop duplicate rows of the same parent Religious Order (becuase there are more than 1 local community of a particular order)
acc_religious_parents.drop_duplicates('Order Full Name', inplace=True)

# How many remaining rows after dropping duplicates?
print(acc_religious_parents.shape)

# Rename columns
acc_religious_parents = acc_religious_parents.rename(columns={
    'Order Full Name': 'Description'
    })

# Drop NA
acc_religious_parents.fillna('', inplace=True)

acc_religious_parents


(62, 6)


<ipython-input-43-7d435f9b98d3>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_religious['Archdpdx_Migration_Id__c'] = acc_religious['Order Full Name'].apply(
<ipython-input-43-7d435f9b98d3>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_religious_parents.drop_duplicates('Order Full Name', inplace=True)


,Description,Name,mbfc__Abbreviation__c,mbfc__Religious_Suffix__c,mbfc__Type_Members__c,Archdpdx_Migration_Id__c
186,Societas Iesu,Colombiere Jesuit Community,Jesuits,SJ,Men,societasiesu
187,Ordo Cisterciensis Strictioris Observantiae,Abbey of Our Lady of Guadalupe,Trappists,OCSO,Men,ordocisterciensisstrictiorisobservantiae
189,Ordo Sancti Benedicti,Benedictine Monks of Mount Angel Abbey,Benedictines,OSB,Men,ordosanctibenedicti
190,Misioneros del Espíritu Santo,Missionaries of the Holy Spirit Provincial House,"Missionaries of the Holy Spirit, Christ the Pr...",MSpS,Men,misionerosdelespíritusanto
191,Apostles of Jesus,Apostles of Jesus,Apostles of Jesus,AJ,Men,apostlesofjesus
...,...,...,...,...,...,...
249,Fraternità san Carlo Borromeo,Priestly Fraternity of the Missionaries of St....,Fraternity of St. Charles,FSCB,Men,fraternitàsancarloborromeo
250,"Sons of Mary, Mother of Mercy","Sons of Mary, Mother of Mercy","Sons of Mary, Mother of Mercy",SMMM,Men,"sonsofmary,motherofmercy"
251,Society of the Divine Word,Society of the Divine Word,Society of the Divine Word,SVD,Men,societyofthedivineword
252,Society of the Divine Saviour,Society of the Divine Saviour,Society of the Divine Saviour,SDS,Men,societyofthedivinesaviour


In [44]:
acc_religious_parents['Religious_Type__c'] = 'Congregation'

In [45]:
#TODO: 'Religious Order', 'Secular Order', 'Diocesan Order', 'Pontifical Order'

In [46]:
# @title  Set recordType to 'Religious'

#
religious_recordtype_id = df_sf_recordTypes.loc[
    (df_sf_recordTypes['DeveloperName'] == 'Religious') & (df_sf_recordTypes['SobjectType'] == 'Account'),
    'Id'
    ].iloc[0]  # Use .iloc[0] to get the first item if you're expecting exactly one match

print(religious_recordtype_id)

acc_religious_parents['RecordTypeId'] = religious_recordtype_id

acc_religious_parents.sample(10)

012Dx0000009TK0IAM


,Description,Name,mbfc__Abbreviation__c,mbfc__Religious_Suffix__c,mbfc__Type_Members__c,Archdpdx_Migration_Id__c,Religious_Type__c,RecordTypeId
219,Congregation of Our Lady of Charity of the Goo...,Good Shepherd Sisters,Good Shepherd Sisters,RGS,Women,congregationofourladyofcharityofthegoods,Congregation,012Dx0000009TK0IAM
202,Congregatio a Sancta Cruce,Congregation of the Holy Cross,Holy Cross,CSC,Men,congregatioasanctacruce,Congregation,012Dx0000009TK0IAM
213,Lovers of Thuthiem Holy Cross Sisters,Thu Thiem Sisters,Thu Thiem Sisters,LHC,Women,loversofthuthiemholycrosssisters,Congregation,012Dx0000009TK0IAM
220,Sisters of the Holy Names of Jesus and Mary U....,Sisters of the Holy Names of Jesus and Mary,Holy Names Sisters,SNJM,Women,sistersoftheholynamesofjesusandmaryu.s.-,Congregation,012Dx0000009TK0IAM
239,Missionary Oblates of Mary Immaculate,Missionary Oblates of Mary Immaculate,Oblates of Mary Immaculate,OMI,Men,missionaryoblatesofmaryimmaculate,Congregation,012Dx0000009TK0IAM
198,Ordo Carmelitarum Discalceatorum,Carmelite House of Studies,Discalced Carmelite Friars,OCD,Men,ordocarmelitarumdiscalceatorum,Congregation,012Dx0000009TK0IAM
209,Congregación de Oblatas de Santa Marta,Oblates of St. Martha,Servites,OSM,Women,congregacióndeoblatasdesantamarta,Congregation,012Dx0000009TK0IAM
221,Sisters of Jesus the Saviour,Sisters of Jesus the Saviour,Sisters of Jesus the Saviour,SJS,Women,sistersofjesusthesaviour,Congregation,012Dx0000009TK0IAM
201,Ordo Fratrum Minorum Province of Saint Barbara,Franciscan Friars,Franciscans,OFM,Men,ordofratrumminorumprovinceofsaintbarbara,Congregation,012Dx0000009TK0IAM
249,Fraternità san Carlo Borromeo,Priestly Fraternity of the Missionaries of St....,Fraternity of St. Charles,FSCB,Men,fraternitàsancarloborromeo,Congregation,012Dx0000009TK0IAM


In [47]:
# @title Send to CSV
acc_religious_parents.to_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/religious_order_staging.csv', encoding='utf-8-sig')

In [48]:
# @title Upsert to Salesforce
bulk_data = []
for row in acc_religious_parents.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)

if run_upserts == 'True':
    religious_order_upsert = sf.bulk.Account.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
    df_rel_order_upsert = pd.DataFrame(religious_order_upsert)

df_rel_order_upsert

,success,created,id,errors
0,True,False,001Dx00001FZmZjIAL,[]
1,True,False,001Dx00001FZmZkIAL,[]
2,True,False,001Dx00001FZmZlIAL,[]
3,True,False,001Dx00001FZmb9IAD,[]
4,True,False,001Dx00001FZmZmIAL,[]
...,...,...,...,...
57,True,False,001Dx00001FZmabIAD,[]
58,True,False,001Dx00001FZmacIAD,[]
59,True,False,001Dx00001FZmadIAD,[]
60,True,False,001Dx00001FZmaeIAD,[]


In [49]:
# Generate an Errors log
import csv

keys = religious_order_upsert[0].keys()

with open('religious_order_results', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, keys)
    writer.writeheader()
    writer.writerows(religious_order_upsert)

# TODO: Convert this into a UDF

In [50]:
# @title get SF Accounts
get_all_rel_accounts = f"Select id, Name, RecordTypeId, Type, Archdpdx_Migration_Id__c from Account where RecordTypeID = '{religious_recordtype_id}'"

print(religious_recordtype_id)

# get list of records, add to dataframe
sf_accounts = sf.query(get_all_rel_accounts)
df_sf_accounts = pd.DataFrame(sf_accounts['records'])
df_sf_accounts = df_sf_accounts.drop(columns = 'attributes')

df_sf_accounts

012Dx0000009TK0IAM


,Id,Name,RecordTypeId,Type,Archdpdx_Migration_Id__c
0,001Dx00001DOGQsIAP,Sociedad San Juan,012Dx0000009TK0IAM,None,None
1,001Dx00001DLKN0IAP,"Saint John Society, Corvallis (SSJ)",012Dx0000009TK0IAM,None,None
2,001Dx00001FZmbAIAT,Sisters of Charity of the Blessed Virgin Mary,012Dx0000009TK0IAM,None,sistersofcharityoftheblessedvirginmary
3,001Dx00001FZmbBIAT,Society of the Missionaries of St. Francis Xavier,012Dx0000009TK0IAM,None,societyofthemissionariesofst.francisxavi
4,001Dx00001FZmbCIAT,Society of Our Lady of the Most Holy Trinity,012Dx0000009TK0IAM,None,societyofourladyofthemostholytrinity
...,...,...,...,...,...
132,001Dx00001FZmjXIAT,"Society of the Divine Saviour, Rome, Italy (SDS)",012Dx0000009TK0IAM,None,Religious_78
133,001Dx00001FZmjYIAT,"Society of Our Lady of the Most Holy Trinity, ...",012Dx0000009TK0IAM,None,Religious_79
134,001Dx00001FZmjZIAT,"Community of St. Thomas More, Eugene (OP)",012Dx0000009TK0IAM,None,Religious_80
135,001Dx00001FZmjaIAD,"Saint Benedict Lodge, McKenzie Bridge (OP)",012Dx0000009TK0IAM,None,Religious_81


In [51]:
religious_order_mapping = df_sf_accounts.set_index('Archdpdx_Migration_Id__c')['Id'].to_dict()
# religious_order_mapping

### D) Religious Communities

In [52]:
"""
- 'acc_religious' DF: create unique_id of religious parents
- create 'acc_religious_orders' DF , upsert into SF
- extract accounts from Salesforce, create dict (external_ID : account_ID)
- map parent ids onto religious child accounts DF in main DF
- 'acc_religious' > staging DF ('acc_religious')
    - drop unnecessary columns
    - upsert create DF of religious children, upsert into SF with
"""

"\n- 'acc_religious' DF: create unique_id of religious parents\n- create 'acc_religious_orders' DF , upsert into SF\n- extract accounts from Salesforce, create dict (external_ID : account_ID)\n- map parent ids onto religious child accounts DF in main DF\n- 'acc_religious' > staging DF ('acc_religious')\n    - drop unnecessary columns\n    - upsert create DF of religious children, upsert into SF with\n"

In [53]:
acc_religious_staging = (acc_religious
                         .rename(columns={'Archdpdx_Migration_Id__c' : 'Parent_Archdpdx_Migration_Id__c'})
)

acc_religious_staging['ParentId'] = acc_religious_staging['Parent_Archdpdx_Migration_Id__c'].map(religious_order_mapping)

In [54]:
# Enrich the data

acc_religious_staging['Religious_Type__c'] = 'Local Community'
acc_religious_staging['Archdpdx_Migration_Id__c'] = 'Religious_' + acc_religious_staging['Record Number'].astype('str')
acc_religious_staging['RecordTypeId'] = religious_recordtype_id
acc_religious_staging.drop(columns='Name', inplace=True)
acc_religious_staging.rename(columns={
    'Name, City': 'Name'
}, inplace=True)

acc_religious_staging.sample(5)

,Record Number,AccountRecordType,Name,Parish Name,Archdiocese Assigns Clergy,Locator Description,BillingStreet,BillingStreet2,BillingCity,BillingState,...,Mailing Address Zip,Vicariate Name,Mailing Address City2,mbfc__Church_Type__c,Parent_Archdpdx_Migration_Id__c,RecordTypeId,Job_Id__c,ParentId,Religious_Type__c,Archdpdx_Migration_Id__c
235,61,Religious,Society of Christ Fathers Province in the Unit...,NaN,No,NaN,Society of Christ,786 West Sunset Ave,Lombard,IL,...,NaN,NaN,NaN,NaN,towarzystwochrystusowe,012Dx0000009TK0IAM,45,001Dx00001FZmaOIAT,Local Community,Religious_61
230,54,Religious,"Society of Mary, Corvallis (SdM)",NaN,No,NaN,540 NW 9th St,NaN,Corvallis,OR,...,NaN,NaN,NaN,NaN,sociedaddemaría,012Dx0000009TK0IAM,45,001Dx00001FZmaKIAT,Local Community,Religious_54
212,36,Religious,"Holy Spirit Sisters, Portland (ALCS)",NaN,No,NaN,2736 NE 54th Ave,NaN,Portland,OR,...,NaN,NaN,NaN,NaN,apostoliclifecommunityofsisters,012Dx0000009TK0IAM,45,001Dx00001FZma3IAD,Local Community,Religious_36
202,23,Religious,"Congregation of the Holy Cross, Portland (CSC)",NaN,No,NaN,5410 N Strong St,NaN,Portland,OR,...,NaN,NaN,NaN,NaN,congregatioasanctacruce,012Dx0000009TK0IAM,45,001Dx00001FZmZvIAL,Local Community,Religious_23
204,27,Religious,"Servite Friars, Portland (OSM)",NaN,No,8719 NE Beech St,Monastery of Our Sorrowful Mother,PO Box 20008,Portland,OR,...,NaN,NaN,NaN,NaN,ordoservorumbeataemariaevirginis,012Dx0000009TK0IAM,45,001Dx00001FZmZxIAL,Local Community,Religious_27


In [55]:
acc_religious_staging_2 = acc_religious_staging[[
    'Name',
    'RecordTypeId',
    'Religious_Type__c',
    'BillingStreet',
    'BillingCity',
    'BillingState',
    'BillingPostalCode',
    'BillingCountry',
    'Phone',
    'Fax',
    'mbfc__Email__c',
    'Website',
    'mbfc__Abbreviation__c',
    'mbfc__Religious_Suffix__c',
    'mbfc__Type_Members__c',
    'Description',
    'Job_Id__c',
    'ParentId',
    'Archdpdx_Migration_Id__c'
    ]]

acc_religious_staging_2.sample(5)

,Name,RecordTypeId,Religious_Type__c,BillingStreet,BillingCity,BillingState,BillingPostalCode,BillingCountry,Phone,Fax,mbfc__Email__c,Website,mbfc__Abbreviation__c,mbfc__Religious_Suffix__c,mbfc__Type_Members__c,Description,Job_Id__c,ParentId,Archdpdx_Migration_Id__c
226,"Sisters of St. Francis, Lake Oswego (OSF)",012Dx0000009TK0IAM,Local Community,843 13th Ave N,Clinton,IA,52732,NaN,503-657-0109,NaN,NaN,http://www.clintonfranciscans.com/,Sisters of St. Francis,OSF,Women,"Serving Our Lady of the Lake Parish, Lake Oswego",45,001Dx00001FZmaGIAT,Religious_50
196,"Félix Rougier House of Studies, Mount Angel (M...",012Dx0000009TK0IAM,Local Community,PO Box 499,Saint Benedict,OR,97373,NaN,503-845-1181,NaN,rougierhos@gmail.com,https://hos.mspsusa.org/home,Missionaries of the Holy Spirit,MSpS,Men,NaN,45,001Dx00001FZmb9IAD,Religious_14
253,"Society of Our Lady of the Most Holy Trinity, ...",012Dx0000009TK0IAM,Local Community,PO Box 4116,Corpus Christi,TX,78469,NaN,NaN,NaN,NaN,https://solt.net/,Society of Our Lady of the Most Holy Trinity,SOLT,Men,NaN,45,001Dx00001FZmbCIAT,Religious_79
227,"Sisters of St. Francis of Philadelphia, Portla...",012Dx0000009TK0IAM,Local Community,609 S Convent Rd,Aston,PA,19014,NaN,610-459-4125,NaN,communications@osfphila.org,https://osfphila.org/,Sisters of St. Francis of Philadelphia,OSF,Women,"Serving Ascension Parish, Cathedral of the Imm...",45,001Dx00001FZmaHIAT,Religious_51
202,"Congregation of the Holy Cross, Portland (CSC)",012Dx0000009TK0IAM,Local Community,5410 N Strong St,Portland,OR,97203,NaN,503-943-8024,503-943-7313,holycrossoffice@up.edu; ministry@up.edu,https://www.holycrossusa.org/,Holy Cross,CSC,Men,Serving the University of Portland; Holy Redee...,45,001Dx00001FZmZvIAL,Religious_23


In [56]:
# Final Cleanup

acc_religious_staging_2 = acc_religious_staging_2.fillna('')

In [57]:
# @title Send to CSV
acc_religious_staging_2.to_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/religious_community_staging.csv', encoding='utf-8-sig')

In [58]:
# @title Upsert to Salesforce
bulk_data = []
for row in acc_religious_staging_2.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)

if run_upserts == 'True':
    religious_community_upsert = sf.bulk.Account.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
    df_rel_community_upsert = pd.DataFrame(religious_community_upsert)

df_rel_community_upsert

,success,created,id,errors
0,True,False,001Dx00001FZmiUIAT,[]
1,True,False,001Dx00001FZmiVIAT,[]
2,True,False,001Dx00001FZmiWIAT,[]
3,True,False,001Dx00001FZmiXIAT,[]
4,True,False,001Dx00001FZmiYIAT,[]
...,...,...,...,...
65,True,False,001Dx00001FZmjWIAT,[]
66,True,False,001Dx00001FZmjXIAT,[]
67,True,False,001Dx00001FZmjYIAT,[]
68,True,False,001Dx00001FZmjZIAT,[]


### E) Religious Superiors

In [59]:
acc_rel_superiors = acc_religious_2[[
    'Name',
    'Major Superior Name',
    'Major Superior Phone',
    'Major Superior Email',
    'Archdpdx_Migration_Id__c']]


acc_rel_superiors['AccountId'] = acc_rel_superiors.Archdpdx_Migration_Id__c.map(religious_order_mapping)

# acc_rel_superiors.sample(5)

<ipython-input-59-d88702d35033>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_rel_superiors['AccountId'] = acc_rel_superiors.Archdpdx_Migration_Id__c.map(religious_order_mapping)


In [60]:
# @title Parse Complex Names
def parse_names(df, column_name):
    # Convert all non-string entries to strings (handling NaN and other data types)
    df[column_name] = df[column_name].fillna('').apply(str)

    # Create a new DataFrame to store the name parts
    name_parts = pd.DataFrame()

    # Parse each name in the column
    name_parts['First Name'] = df[column_name].apply(lambda x: HumanName(x).first if x.strip() != '' else '')
    name_parts['Last Name'] = df[column_name].apply(lambda x: HumanName(x).last if x.strip() != '' else '')
    name_parts['Middle Name'] = df[column_name].apply(lambda x: HumanName(x).middle if x.strip() != '' else '')
    name_parts['Title'] = df[column_name].apply(lambda x: HumanName(x).title if x.strip() != '' else '')
    name_parts['Suffix'] = df[column_name].apply(lambda x: HumanName(x).suffix if x.strip() != '' else '')
    name_parts['Nickname'] = df[column_name].apply(lambda x: HumanName(x).nickname if x.strip() != '' else '')

    # Combine the original DataFrame with the name parts DataFrame
    result_df = pd.concat([df, name_parts], axis=1)
    return result_df



In [61]:
!pip install nameparser
from nameparser import HumanName
from nameparser.config import CONSTANTS

# Add dataset-specific Titles and Suffix constants for parsing
CONSTANTS.titles.add('Very', 'Rev.', 'Very Rev.', 'Sr.')
CONSTANTS.suffix_acronyms.add('FRS', 'OMI', 'OSA', 'OCD', 'OP', 'OC', 'FSE', 'OMV', 'SDB', 'SM', 'SFX', 'SP', 'OP', 'O.S.M', 'SNJM', 'OSF', 'HMRF', 'DD', 'CSJP', 'SDD', 'BVM', 'BVM - President' )


SetManager({'am', 'chmm', 'cpfa', 'cscp', 'lt', 'bvm - president', 'lcmt', 'facep', 'gchs', 'lvo', 'rid', 'hrs', 'kchs/dchs', 'stmieee', 'cst', 'ncto', 'fcela', 'sasm', 'cic', 'cism', 'faicp', 'ei', 'cae', 'cb', 'cpss', 'aqp', 'fca', 'cfce', 'faan', 'iaee', 'mlt', 'sp', 'same', 'cprp', 'dma', 'cpim', 'mcdba', 'psm i', 'apr', 'cwap', 'nicet i', 'kcb', 'mcct', 'lmsw', 'sgm', 'mem', 'cprc', 'cmfo', 'cbs', 'dsc', 'mvo', 'secb', 'aia', 'mc', 'psyd', 'iccm-f', 'asla', 'capa', 'nbcfch', 'si', 'cdmp', 'ncso', 'chpln', 'lpn', 'rrc', 'lp', 'clsd', 'nbcch-ps', 'fws', 'nicet ii', 'dabfm', 'crtt', 'fsdp', 'abpp', 'msa', 'ficf', 'cfc', 'ndtr', 'cxa', 'cet', 'enp', 'clu', 'nbcdch', 'ccie', 'cla', 'mbe', 'cgsp', 'thm', 'ocd', 'lcsw', 'litk', 'cwsp', 'rpa', 'pa-c', 'cpo', 'iaccp', 'ncps', 'gisp', 'psp', 'btcs', 'cgr', 'cpht', 'cisa', 'dvm', 'cbte', 'jp', 'ccp', 'csep', 'aba', 'cig', 'cma', 'bts', 'cpp', 'cfp', 'kcmg', 'oscp', 'uxc', 'hccp', 'prm', 'lg', 'nicet iv', 'kbe', 'gcb', 'abr', 'afc', 'cep', 'c

In [62]:
# Parse Complex Names
acc_rel_superiors_parsed = parse_names(acc_rel_superiors, 'Major Superior Name')

<ipython-input-60-b9e5df43049d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].fillna('').apply(str)


In [63]:
# @title Final cleanup

acc_rel_superiors_staging = acc_rel_superiors_parsed.fillna('')

acc_rel_superiors_staging['Archdpdx_Migration_Id__c'] = acc_rel_superiors_staging['Major Superior Name'].apply(lambda x: x.replace(' ','').lower())

# Rename columns
acc_rel_superiors_staging = acc_rel_superiors_staging.rename(columns={
    'Major Superior Phone': 'Phone',
    'Major Superior Email': 'Email',
    'Title': 'Salutation',
    'First Name': 'FirstName',
    'Middle Name': 'MiddleName',
    'Last Name': 'LastName'
})

# Add job id
acc_rel_superiors_staging['Archdpdx_Job_Id__c'] = curr_job_id

# Drop columns
acc_rel_superiors_staging = acc_rel_superiors_staging.drop(columns=['Name', 'Major Superior Name', 'Nickname'])

# Drop empty rows
acc_rel_superiors_staging = acc_rel_superiors_staging[acc_rel_superiors_staging['LastName'].str.strip() != '']

acc_rel_superiors_staging.sample(10)

,Phone,Email,Archdpdx_Migration_Id__c,AccountId,FirstName,LastName,MiddleName,Salutation,Suffix,Archdpdx_Job_Id__c
245,617-536-4141,office@omvusa.org,"fr.jimwalther,omv",001Dx00001FZmaXIAT,Jim,Walther,,Fr.,OMV,45
200,504-254-9429,domusdeiusa@gmail.com,"veryrev.josephhaidangvu,sdd",001Dx00001FZmZtIAL,Joseph,Vu,Hai Dang,Very Rev.,SDD,45
193,,"P.O. Box 8816 Moshi, Tanzania",rev.charleslyimo,001Dx00001FZmZoIAL,Charles,Lyimo,,Rev.,,45
189,630-897-7215,jbrahill@marmion.org,rightreverendjohnbrahill,001Dx00001FZmZlIAL,John,Brahill,,Right Reverend,,45
191,610-502-1732,ajregionalusa@gmail.com,reverendaugustineidra,001Dx00001FZmZmIAL,Augustine,Idra,,Reverend,,45
215,011 52 55 58 72 20 0,hmrf@misionerasdefatima.org,"candelarianavarroalvarado,hmrf",001Dx00001FZma6IAD,Candelaria,Alvarado,Navarro,,HMRF,45
228,,,"sisterandreanenzel,csjp",001Dx00001FZmaIIAT,Andrea,Nenzel,,Sister,CSJP,45
232,818-626-9193,info@eparchy.org,"bishopa.eilaszaidan,ddmlm,eparchyofourladyofle...",001Dx00001FZmaMIAT,DD,Bishop A. Eilas Zaidan,MLM,,Eparchy of Our Lady of Lebanon of Los Angeles,45
198,909-793-0424,,"fr.matthewwilliams,o.c.d.",001Dx00001FZmZrIAL,Matthew,Williams,,Fr.,O.C.D.,45
214,914-941-7575,maryknoll@mksisters.org,"sisterteresahougnon,mm,president",001Dx00001FZma5IAD,Teresa,Hougnon,,Sister,"MM, President",45


In [64]:
# @title Send to CSV
acc_rel_superiors_staging.to_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/religious_superiors_staging.csv', encoding='utf-8-sig')

In [65]:
# @title Upsert to Salesforce

def find_existing_contact(sf, first_name, last_name):
    query = f"SELECT Id, Archdpdx_Migration_Id__c FROM Contact WHERE FirstName = '{first_name}' AND LastName = '{last_name}'"
    result = sf.query(query)
    return result['records']



bulk_data = []
for row in acc_rel_superiors_staging.itertuples(index=False):
    d = row._asdict()
    existing_contacts = find_existing_contact(sf, d['FirstName'], d['LastName'])
    if existing_contacts:
        # Update existing contact with external ID
        d['Id'] = existing_contacts[0]['Id']
        bulk_data.append(d)
    else:
        bulk_data.append(d)


if run_upserts == 'True':
    religious_superior_upsert = sf.bulk.Contact.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
    df_rel_superior_upsert = pd.DataFrame(religious_superior_upsert)

df_rel_superior_upsert

,success,created,id,errors
0,False,False,None,"[{'statusCode': 'DUPLICATE_VALUE', 'message': ..."
1,True,False,003Dx00000m0jAWIAY,[]
2,True,False,003Dx00000m0jAXIAY,[]
3,True,False,003Dx00000m0jAYIAY,[]
4,False,True,None,"[{'statusCode': 'INVALID_EMAIL_ADDRESS', 'mess..."
5,True,False,003Dx00000m0jAZIAY,[]
6,True,False,003Dx00000m0jAaIAI,[]
7,False,False,None,"[{'statusCode': 'DUPLICATE_VALUE', 'message': ..."
8,True,False,003Dx00000m0jAbIAI,[]
9,True,False,003Dx00000m0jAcIAI,[]


In [66]:
# @title Update Religious Communities with Rel. Superior

# TBD: It would take much less time to simply do this post-migration manually.

# CONTACTS

## Extract

In [67]:
df_contacts = (pd.read_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/People.csv')
               .set_index('Record Number', verify_integrity=True)
               .drop(index='recNum') # Drops the extra row that replicates the labels
               .rename(columns=lambda x: x.replace(' ', '_')) # Remove whitespace in column names
)

#TODO - drop the first row of the df

df_contacts.sample(10)


,Common_Name,Sort_Name,Type(s),Clergy_Status,Religious_Status,Login_ID,Password,Password_Must_be_Changed,Access_Permission,Spouse,...,CARA_Ethnicity,Seminarian_Status,Other_Diaconal_Ministry,Spiritual_Director_Authorized,Link_to_Religious_Community,Place_of_Work,Volunteer_Place,Type_of_Work,Work_Load,Work_Title
Record Number,,,,,,,,,,,,,,,,,,,,,
6,Mrs. Paula Cramer,cramer paula ann,Wife,NaN,NaN,pacramer,3ffd097dfaae7b1097a06f928b7830961080b5d846067b...,Yes,NaN,310,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1230,Rev. William Minkel,minkel william,Priest,Transferred Out,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
3179,Ms. Lizbeth Chavez,chavez lizbeth,Staff,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
576,"Rev. Art Wheeler, CSC",wheeler art,"Priest,Religious",Active,Active,awheeler,84012adf55f036b78f1a6aea0783f1436a15c5e2d49f98...,No,NaN,0,...,NaN,NaN,NaN,NaN,23,University of Portland,NaN,Education,Full Time,Associate Professor
642,"Rev. Francis Nguyen, SDD",nguyen francis xavier vinh van,"Priest,Religious",Transferred Out,Transferred Out,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,21,NaN,NaN,NaN,NaN,NaN
962,Mr. Mark Cach,cach mark,Priest,Laicized,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
3228,Ms. Laura Scofield,scofield laura,Staff,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
419,Deacon Roger Porcella,porcella roger edward,Permanent Deacon,Deceased,NaN,NaN,NaN,NaN,NaN,0,...,Hispanic/Latino,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2544,"Sr. Therese Improgo, OSF",improgo therese,Religious,NaN,Active,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,35,Our Lady of Peace Retreat Center,NaN,Spiritual Direction,Part Time,Spiritual Director


#### Get Photos

In [68]:
import os
import pandas as pd

def list_jpeg_files(directory):
    data = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpeg") or filename.endswith(".jpg"):  # Checking for jpeg files
            full_path = os.path.join(directory, filename)
            data.append({'Filename': filename, 'Full Path': full_path})
    return pd.DataFrame(data)

# Specify your directory
directory = '/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/sql_backup/archdpdx.info backups/public_html/people/graphics/portraits/large'
jpeg_files_df = list_jpeg_files(directory)


In [69]:
# Query for the Library
library_query = "SELECT Id, Name FROM ContentWorkspace WHERE Name = 'ADPDX Person Profile Photos'"
library_result = sf.query(library_query)

# Check if the library exists and get its ID
if library_result['records']:
    library_id = library_result['records'][0]['Id']
    print(f"Library ID: {library_id}")

    # Query for the Folder within the Library
    folder_query = f"SELECT Id, Name FROM ContentFolder WHERE ParentContentFolderId = '{library_id}'"
    folder_result = sf.query(folder_query)

    # Check if the folder exists and get its ID
    if folder_result['records']:
        folder_id = folder_result['records'][0]['Id']
        print(f"Folder ID: {folder_id}")
    else:
        print("Folder 'Large JPEGs' not found in the library.")
else:
    print("Library 'ADPDX Person Profile Photos' not found.")

Library ID: 058Dx0000006ZYKIA2
Folder 'Large JPEGs' not found in the library.


In [70]:
folder_result

OrderedDict([('totalSize', 0), ('done', True), ('records', [])])

## Analysis

Here we check the various columns and their types, count where values exist, count of unique values, sample data, etc.

DF shape:
- 142 columns
- 3017 rows

In [71]:
# Check the original shape of the imported CSV
print(f"Shape of original data set: {df_contacts.shape}")

# export to csv a list of the contact fields with count, unique, top, freq
contacts_describe = df_contacts.describe(include='all').transpose()
contacts_describe.to_csv(f'/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/analysis/contacts_describe.csv')

df_contacts.describe(include='all').transpose()  #initial analysis of the Contacts table

Shape of original data set: (3016, 141)


,count,unique,top,freq
Common_Name,3016,3011,Ms. Leslie Jones,2
Sort_Name,3016,3009,nguyen anthony,3
Type(s),3016,29,Staff,1139
Clergy_Status,1138,8,Transferred Out,462
Religious_Status,902,4,Active,456
...,...,...,...,...
Place_of_Work,269,133,Mount Angel Abbey,37
Volunteer_Place,54,47,Mary’s Woods,4
Type_of_Work,276,117,Pastoral Ministry,30
Work_Load,262,2,Full Time,230


In [72]:
unique_languages = df_contacts['Languages'].unique()
unique_languages

array([nan, 'English,Spanish', 'Igbo', 'English, Spanish',
       'Spanish, Mayaqeqchi', 'Spanish (Mass only)',
       'Latin Mass and written translation. Read French, Italian, Spanish.',
       'Spanish', 'Hindi, Konkani, Tamil',
       'French (fluent), Spanish (beginner), Latin (beginner)',
       'German, Spanish, Italian, French', 'Kiswahili, Kichagga',
       'Spanish (English is second language)',
       'German, Spanish, Italian, Latin Mass',
       'English, Spanish, Italian', 'Spanish, Italian', 'English',
       'Bicolango, Tagalog, Spanish', 'Spanish, Italian, Latin Mass',
       'Italian', 'Tagalog, English, Spanish',
       'French, Italian, Aramaic (modern), Spanish', 'Vietnamese',
       'German, Spanish', 'English,Spanish,Italian',
       'Conversant in Italian and Spanish, some facility with Latin and German',
       'English, Spanish, Latin Mass', 'Italian, Spanish',
       'Konkani, Hindi, Marathi, Spanish',
       'Tagalog, Bicol, Spanish (Mass only)', 'Spanish, E

In [73]:
import re
import numpy as np


def deduplicate_languages(list_languages):
    # Define a regular expression pattern to match periods and punctuation
    punctuation_pattern = r'[.,!?;:"]'

    # Flatten the array and filter out NaN values
    flattened_languages = [re.sub(punctuation_pattern, '', lang) for sublist in list_languages if pd.notna(sublist) for lang in sublist.split(',')]

    # Deduplicate the list of languages
    unique_languages = list(set(flattened_languages))

    return unique_languages


# Example usage:
unique_languages = deduplicate_languages(unique_languages)
print(unique_languages)


['', 'Vietnamese', ' Maya Q’eqchi’', 'Chuukese', 'Latin Mass and written translation Read French', ' Englsih', 'Kisii', 'Vietnamese (Mass only)', ' English', ' Swahili Mass', ' Croatian', 'Latin Mass', ' a little Spanish(not conversational', ' Mayaqeqchi', ' Chamorro', ' Spanish (small bits)', 'Spanish', 'Conversant in Italian and Spanish', ' Hebrew', ' Telugu', ' Spanish', ' Tagalog', 'Tamil (Indian)', 'German', 'English', ' Spanish (beginner)', 'Portuguese', ' Konkawin', ' Little Spanish', ' Russian', 'Chagga', ' Latin', ' Kichagga', ' Greek', 'Korean', ' some facility with Latin and German', ' but can do rituals)', 'Italian', 'Kiswahili', 'Tagalog', 'French (small bits)', ' Portuguese', ' Vietnamese', ' French', ' German', 'French (fluent)', 'Polish', ' Kiswahili', 'Latin', ' Italian', ' Konkani', ' Spanish Mass', ' Tamil', ' Latin (beginner)', ' Aramaic (modern)', 'Tamil', 'Spanish (Mass only)', ' Marathi', 'Afrikaans', 'Crijolle', ' Hebrew (reading)', ' Hindi', ' Latin Mass', 'Kon

## Transform

In [74]:
# init list of columns NOT to be loaded as Contact attributes
misc_columns_to_drop = [
    'Password',
    'Password_Must_be_Changed',
    'Common_Name',
    'Sort_Name',
    'Private_Address_Province'
]

affiliation_columns = [
    'Seminarian_Student_Debt',
    'Seminarian_Medical_Benefits',
    'Baptism_Date',
    'Place_of_Baptism',
    'Confirmation_Date',
    'Place_of_Confirmation',
    'Received_Date',
    'Parish_of_Record',
    'Marriage_Date',
    'Place_of_Marriage',
    'Date_of_First_Vows',
    'Date_of_Final_Vows',
    'Accepted_to_Formation_Date',
    'Reader_Date',
    'Acolyte_Date',
    'Candidacy_Date',
    'Formation_Withdrawn_Date',
    'Formation_Deferred_Date',
    'Formation_Terminated_Date',
    'Terminate_or_Defer_Note',
    'Bachelor_Degree_Year',
    'Bachelor_Degree_Type',
    'Bachelor_Degree_Institution',
    'Graduate_1_Degree_Institution',
    'Graduate_1_Degree_Type',
    'Graduate_1_Degree_Year',
    'Graduate_2_Degree_Institution',
    'Graduate_2_Degree_Type',
    'Graduate_2_Degree_Year',
    'Graduate_3_Degree_Institution',
    'Graduate_3_Degree_Type',
    'Graduate_3_Degree_Year',
    'Graduate_4_Degree_Institution',
    'Graduate_4_Degree_Type',
    'Graduate_4_Degree_Year',
    'CARA_Highest_Ed_Level',
    'Diaconal_Ordination_Date',
    'Diaconal_Ordination_Place',
    'Diaconal_Ordination_Prelate',
    'Presbyteral_Ordination_Date',
    'Presbyteral_Ordination_Place',
    'Presbyteral_Ordination_Prelate',
    'Episcopal_Ordination_Date',
    'Episcopal_Ordination_Place',
    'Episcopal_Ordination_Prelate',
    'Ordination_Diocese',
    'Incardinated_From_Date',
    'Incardinated_From_Diocese',
    'Incardinated_Now',
    'Serving_Now',
    'Excardinated_To_Diocese',
    'Excardinated_To_Date',
    'Letter_of_Good_Standing_Date',
    'Religious_In_Archdiocese_Date',
    'Faculties',
    'Faculties_Granted_Date',
    'Faculties_Restricted_Date',
    'Faculties_Withdrawn_Date',
    'Last_Retreat_Date',
    'Last_Educ_Requirement_Date',
    'Policy_Manual_Acknowledgement_Date',
    'Harassment_Prevention_Course_Date',
    'Standards_of_Conduct_Date',
    'Last_Background_Check_Date',
    'Last_Child_Protection_Training_Date',
    'Out_of_Diocese_Date',
    'Senior_Status_Date',
    'Laicized_Date',
    'Coverage_Availability',
    'Advanced_Directive_Date',
    'End_of_Life_Plan_Date',
    'Will_Date',
    'Will_Note',
    'CIC_489_File',
    'Registered_Parish',
    'CARA_Ethnicity',
    'Seminarian_Status',
    'Other_Diaconal_Ministry',
    'Spiritual_Director_Authorized',
    'Link_to_Religious_Community',
    'Place_of_Work',
    'Volunteer_Place',
    'Type_of_Work',
    'Work_Load',
    'Work_Title'
]

In [75]:
# These fields need to be KEPT but while building the SF upsert flow these are dropped temporarily until mapping logic is included.
# TODO

fields_not_yet_mapped = [
    'Spouse',
    'Father_Full_Name',
    'Mother_Full_Maiden_Name',
    'Mailing_Address_2',
    'Mailing_Address_Province',
    'Private_Address_2',
    'Nickname',
    'Preferred_Address',
    'Private_Address__Street__s',
    'Private_Address__City__s',
    'Private_Address__State__s',
    'Private_Address__PostalCode__s',
    'Private_Address__Country__s',
    'Social_Security_Account_Number__c',  # The data is encrypted
    'Languages',  # Picklist is restricted, in MFC package. Needs unrestricting before I can migrate data.
    'Preferred_Email',
    'Preferred_Phone'

]

In [76]:
# UDF to combine multiple Mailing Street Address lines into one

def combine_addresses(row, *columns):
    address_parts = []
    for col in columns:
        address_parts.append(row[col])
    return 'CHAR(10)'.join(address_parts)


In [77]:
df_contact_staging = (df_contacts
                      .drop(columns='Salutation')
                      .rename(columns={
                          'Clergy_Status' : 'ADPDX_Clergy_Status__c',
                          'Religious_Status' : 'ADPDX_Religious_Status__c',
                          'Login_ID' : 'ADPDX_Login_ID__c',
                          'Access_Permission': 'ADPDX_Access_Permission__c',
                          'Title': 'Salutation',
                          'Christian_Name': 'FirstName',
                          'Middle_Name(s)': 'MiddleName',
                          'Surname': 'LastName',
                          'Suffix': 'Suffix',
                          #Mailing_Address & Mailing_Address_2
                          'Mailing_Address' : 'MailingStreet',
                          'Mailing_Address_City': 'MailingCity',
                          'Mailing_Address_State': 'MailingState',
                          #'Mailing_Address_Province': 'MailingProvince'
                          'Mailing_Address_Postal_Code': 'MailingPostalCode',
                          'Mailing_Address_Country': 'MailingCountry',
                          'Private_Address': 'Private_Address__Street__s',
                          #'Private Address 2' : 'Private_Address__Street__s,
                          'Private_Address_City': 'Private_Address__City__s',
                          'Private_Address_State': 'Private_Address__StateCode__s',
                          'Private_Address_Postal_Code': 'Private_Address__PostalCode__s',
                          'Private_Address_Country': 'Private_Address__CountryCode__s',
                          # 'Preferred_Address'
                          'Work_Phone': 'npe01__WorkPhone__c',
                          'Home_Phone': 'HomePhone',
                          'Cell_Phone': 'MobilePhone',
                        #   'Preferred_Phone': 'npe01__PreferredPhone__c',
                          # IF Preferred phone contains, 'do not publish'
                          'Work_Email' : 'npe01__WorkEmail__c',
                          'Archdiocesan_Email': 'npe01__AlternateEmail__c',
                          'Home_Email': 'npe01__HomeEmail__c',
                        #   'Preferred_Email': 'npe01__Preferred_Email__c',
                          # IF Preferred email contains 'do not publish''
                          'Directory_Include': 'Directory_Include__c',
                          'Directory_Include_Middle_Name': 'Directory_Include_Middle_Name__c',
                          'Directory_Include_Suffix': 'Directory_Include_Suffix__c',
                          'Suppress_From_Reports': 'Suppress_From_Reports__c',
                          'Send_Group_Mail_and_Email': 'Send_Group_Mail_and_Email__c',
                          'Birth_Date': 'Birthdate',
                          'Place_of_Birth': 'mbfc__Place_of_Birth__c',
                          'Foreign_Born': 'Foreign_Born__c',
                          'Foreign_Citizenship': 'Foreign_Citizenship__c',
                          'Immigration_Status': 'Immigration_Status__c',
                          'Passport/Visa_Expiration_Date': 'Passport_Visa_Expiration_Date__c',
                          'Social_Security_Account_Number': 'Social_Security_Account_Number__c',
                          'Deceased_Date': 'mbfc__Date_of_Death__c',
                          'Languages': 'Languages__c'
                          })
                      .assign(Bi_Ritual__c=lambda x: x['Type(s)'].str.contains('Biritual'))
                      .assign(Non_Latin_Rite__c=lambda x: x['Type(s)'].str.contains('Non-Latin Rite'))
                      .assign(Archdpdx_Migration_Id__c=lambda x: x.index)
                    #   .assign(Mailing_Address=lambda row: combine_addresses(row, 'Mailing_Address', 'Mailing_Address_2'), axis=1)
                      .drop(columns=misc_columns_to_drop)
                      .drop(columns=affiliation_columns)
                      .drop(columns=fields_not_yet_mapped)

        )


df_contact_staging.sample(10)

KeyError: "['Private_Address__State__s', 'Private_Address__Country__s', 'Languages'] not found in axis"

### Private Address Handling

In [ ]:
df_contact_staging.loc[:,'Private_Address__Street__s':'Private_Address__CountryCode__s'][~df_contact_staging['Private_Address__StateCode__s'].isna()]

In [ ]:
df_contact_staging['Private_Address__CountryCode__s'] = df_contact_staging.apply(lambda row: 'United States' if pd.notnull(row['Private_Address__StateCode__s']) and pd.isnull(row['Private_Address__CountryCode__s']) else row['Private_Address__CountryCode__s'], axis=1)

### Handle Boolean Fields

In [ ]:
boolean_columns_to_convert = ['Foreign_Born__c', 'Directory_Include__c', 'Directory_Include_Middle_Name__c', 'Directory_Include_Suffix__c',
       'Suppress_From_Reports__c', 'Send_Group_Mail_and_Email__c', ]

df_contact_staging[boolean_columns_to_convert] = df_contact_staging[boolean_columns_to_convert].replace({'Yes': True, 'No': False})


In [ ]:
df_contact_staging[boolean_columns_to_convert] = df_contact_staging[boolean_columns_to_convert].fillna(False)

df_contact_staging[boolean_columns_to_convert].sample(5)

### Set Contact Record Type

In [ ]:
# Set Record Type

# Go down row by row and check the 'Type(s)' columns, check for certain words that are keys in a dictionary, and
# the that row's 'Type(s)' field contains a string that is in the a key in a dictionary the update another columns
# called 'ContactRecordType' with the paired value.

contact_type_map = {
    'Bishop': 'Priest',
    'Diaconate': 'Lay_Person',
    'Permanent Deacon': 'Permanent_Deacon',
    'Priest': 'Priest',
    'Staff': 'Lay_Person',
    'Seminarian': 'Lay_Person',
    'Wife': 'Lay_Person',
    'Religious': 'Religious',
    'Seminary Applicant': 'Lay_Person',
    'Transitional Deacon': 'Priest',
    'Archive': 'Lay_Person'
}

def update_contact_record_type(row):
    for key, value in contact_type_map.items():
        if key in row['Type(s)']:
            return value
    return None

df_contact_staging['ContactRecordType'] = df_contact_staging.apply(update_contact_record_type, axis=1)

In [ ]:
# Map in the RecordTypeIDs
df_contact_staging['RecordTypeID'] = df_contact_staging['ContactRecordType'].map(record_types_mapping)

In [ ]:
# Check for any Contacts who are missing a RecordTypeId
df_contact_staging[df_contact_staging['RecordTypeID'].isna()]

### Email Validation

In [ ]:
# !pip install email_validator
from email_validator import validate_email, EmailNotValidError

# function that validates an email and if it is invalid it returns nothing
def validate_email_address(email):
    if isinstance(email, float):
        return None
    try:
        v = validate_email(email)
        return v.email
    except EmailNotValidError:
        return None


In [ ]:
# init a list of Email columns
# email_columns = ['npe01__HomeEmail__c', 'npe01__WorkEmail__c', 'npe01__AlternateEmail__c']

# df_contact_staging[email_columns] = df_contact_staging[email_columns].applymap(validate_email_address)

### Final Dataframe Cleanup

In [ ]:
# drop columns that are no longer needed
del df_contact_staging['Type(s)']
del df_contact_staging['ContactRecordType']

In [ ]:
# clean up all NaN values

df_contact_staging = df_contact_staging.fillna('')

## Load

In [ ]:
df_contact_staging['Archdpdx_Job_Id__c'] = curr_job_id

In [ ]:
# generate CSV for manual loading
df_contact_staging.to_csv(f'/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/df_contacts_staging.csv', encoding='utf-8-sig')
df_contact_staging.to_csv('contacts_staging.csv', encoding='utf-8-sig')


In [ ]:
# upsert Contact records into SF using Bulk api

from simple_salesforce.exceptions import SalesforceMalformedRequest

bulk_data = []
for row in df_contact_staging.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)

try:
    # Attempt to upsert Contact records into SF using Bulk API
    contact_upsert = sf.bulk.Contact.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=True)
    contact_upsert_results = pd.DataFrame(contact_upsert)
except SalesforceMalformedRequest as e:
    # If a SalesforceMalformedRequest error occurs, print the error message and response content
    print(f"SalesforceMalformedRequest error: {e}")
    print(f"Response content: {e.content}")

In [ ]:
# Print upsert results to local file

keys = contact_upsert[0].keys()

with open('contact_results', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, keys)
    writer.writeheader()
    writer.writerows(contact_upsert)


# CONTACT > REGISTER ENTRIES

In [ ]:
import pandas as pd

# Load CSV
df = (pd.read_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/People.csv')
               .rename(columns=lambda x: x.replace(' ', '_')) # Remove whitespace in column names
               .drop(index=0) # Drops the extra row that replicates the labels
)

df

In [ ]:
# @title Parse Sacrament & Notation Types

#TODO: Baptisms should be set to 'Notice of Baptism', not 'Baptism

import pandas as pd

# Define the structure of your column sets with correct attribute names
column_sets = [
    {'date': 'Baptism_Date', 'place': 'Place_of_Baptism', 'notation_type': 'Proof of Baptism'},
    {'date': 'Confirmation_Date', 'place': 'Place_of_Confirmation', 'notation_type': 'Notice of Confirmation'},
    {'date': 'Received_Date', 'place': 'Parish_of_Record', 'notation_type': 'Notation of Profession of Faith'},
    {'date': 'Marriage_Date', 'place': 'Place_of_Marriage', 'notation_type': 'Notice of Matrimony'},
    {'date': 'Diaconal_Ordination_Date', 'place': 'Diaconal_Ordination_Place', 'prelate': 'Diaconate_Ordination_Prelate', 'notation_type': 'Notice of Holy Orders', 'ordination_type': 'Diaconate'},
    {'date': 'Presbyteral_Ordination_Date', 'place': 'Presbyteral_Ordination_Place', 'prelate': 'Presbyteral_Ordination_Prelate', 'notation_type': 'Notice of Holy Orders', 'ordination_type': 'Presbyteral'},
    {'date': 'Episcopal_Ordination_Date', 'place': 'Episcopal_Ordination_Place', 'prelate': 'Episcopal_Ordination_Prelate', 'notation_type': 'Notice of Holy Orders', 'ordination_type': 'Episcopal'}
]

# New DataFrame for entries
register_entries = pd.DataFrame(columns=['RecordNumber', 'mbfc__Register_Entry_Type__c', 'mbfc__Type__c', 'mbfc__Notation_Type__c', 'mbfc__Ordination_Type__c', 'Date', 'Place', 'Prelate'])
new_entries = []  # List to store entries before final concatenation

# Processing rows
for row in df.itertuples():
    for column_set in column_sets:
        date_value = getattr(row, column_set['date'], None)
        if pd.notna(date_value):  # Check if date field is not NaN
            entry = {
                'RecordNumber': getattr(row, 'Record_Number', None),
                'Date': date_value,
                'Place': getattr(row, column_set['place'], None)
            }
            # Add Prelate if applicable
            if 'prelate' in column_set:
                entry['Prelate'] = getattr(row, column_set['prelate'], None)

            # Set 'mbfc__Register_Entry_Type__c', and conditionally add 'mbfc__Type__c' or 'mbfc__Notation_Type__c'
            if 'sacrament_type' in column_set:
                entry['mbfc__Type__c'] = column_set['sacrament_type']
                entry['mbfc__Register_Entry_Type__c'] = 'Sacrament'
            if 'notation_type' in column_set:
                entry['mbfc__Notation_Type__c'] = column_set['notation_type']
                entry['mbfc__Register_Entry_Type__c'] = 'Notation'

            # Handle ordination type specific updates
            if 'ordination_type' in column_set:
                entry['mbfc__Ordination_Type__c'] = column_set['ordination_type']

            new_entries.append(entry)

# Concatenate all new entries to the DataFrame at once
if new_entries:
    register_entries = pd.concat([register_entries, pd.DataFrame(new_entries)], ignore_index=True)

print(f"Total records added: {len(register_entries)}")

# Optionally, save the new DataFrame to a CSV
register_entries.to_csv('Register_Entries.csv', index=False)

# Display the DataFrame
register_entries.sample(10)


In [ ]:
from nameparser import HumanName
from nameparser.config import CONSTANTS

# Add dataset-specific Titles and Suffix constants for parsing
CONSTANTS.titles.add('Very', 'Rev.', 'Very Rev.', 'Sr.', 'Most Rev.')
CONSTANTS.suffix_acronyms.add('FRS', 'J.C.L.', 'J.C.L., D.D.', 'D.D.', 'OMI', 'OSA', 'OCD', 'OP', 'OC', 'FSE', 'OMV', 'SDB', 'SM', 'SFX', 'SP', 'OP', 'O.S.M', 'SNJM', 'OSF', 'HMRF', 'DD', 'CSJP', 'SDD', 'BVM', 'BVM - President', 'SJ', 'SL', 'IX', 'SSJ', 'J.C.L.', 'J.C.L', 'OFM', 'MSpS', 'Fco.' )


def parse_name(name):
    if pd.isna(name):  # Checks if the name is NaN or None
        return {
            'Salutation': '',
            'FirstName': '',
            'MiddleName': '',
            'LastName': '',
            'Suffix': ''
        }
    else:
        name = HumanName(name)
        return {
            'Salutation': name.title,
            'FirstName': name.first,
            'MiddleName': name.middle,
            'LastName': name.last,
            'Suffix': name.suffix
        }

# Apply the parsing function only where 'Prelate' exists and is not NaN
for entry in new_entries:
    if 'Prelate' in entry and pd.notna(entry['Prelate']):
        parsed_name = parse_name(entry['Prelate'])
        entry.update(parsed_name)

# Ensure the DataFrame creation from new_entries includes checks for existence of keys:
register_entries = pd.DataFrame(new_entries)
if 'Prelate' in register_entries.columns:
    register_entries['Salutation'] = register_entries['Prelate'].apply(lambda x: parse_name(x)['Salutation'] if pd.notna(x) else '')
    register_entries['FirstName'] = register_entries['Prelate'].apply(lambda x: parse_name(x)['FirstName'] if pd.notna(x) else '')
    register_entries['MiddleName'] = register_entries['Prelate'].apply(lambda x: parse_name(x)['MiddleName'] if pd.notna(x) else '')
    register_entries['LastName'] = register_entries['Prelate'].apply(lambda x: parse_name(x)['LastName'] if pd.notna(x) else '')
    register_entries['Suffix'] = register_entries['Prelate'].apply(lambda x: parse_name(x)['Suffix'] if pd.notna(x) else '')


# Display the DataFrame
print(f"Total records added: {len(register_entries)}")
register_entries.sample(10)



In [ ]:
# @title Query Salesforce for existing contacts and create a dictionary for mapping

# from simple_salesforce import Salesforce

# query = """
# SELECT Id, Archdpdx_Migration_ID__c
# FROM Contact
# """
# result = sf.query_all(query)
# contact_map = {rec['Archdpdx_Migration_ID__c']: rec['Id'] for rec in result['records']}


In [ ]:
# Get RecordTypeId for Contact.Priest

priest_contact_recordtype_id = df_sf_recordTypes.loc[
    (df_sf_recordTypes['DeveloperName'] == 'Priest') & (df_sf_recordTypes['SobjectType'] == 'Contact'),
    'Id'
    ].iloc[0]  # Use .iloc[0] to get the first item if you're expecting exactly one match


In [ ]:
# @title Query for Contacts by Names and Createe New Contacts

from simple_salesforce import SFType, SalesforceResourceNotFound

contact = SFType('Contact', sf.session_id, sf.sf_instance)
for index, row in register_entries.iterrows():
    first_name, last_name = row.get('FirstName'), row.get('LastName')

    if pd.isna(first_name) or pd.isna(last_name) or first_name.strip() == '' or last_name.strip() == '':
        # If either first name or last name is missing or empty, skip this row or handle as needed
        print(f"Skipping row {index} due to missing name information.")
        continue

    try:
        # Search for contact by First and Last Name
        query = f"SELECT Id FROM Contact WHERE FirstName = '{first_name}' AND LastName = '{last_name}'"
        result = sf.query(query)
        if result['totalSize'] > 0:
            contact_id = result['records'][0]['Id']
        else:
            # Create a new contact if no match found
            new_contact = {
                'FirstName': first_name,
                'LastName': last_name,
                'Archdpdx_Job_Id__c': curr_job_id,
                'RecordTypeId': priest_contact_recordtype_id
            }
            create_result = contact.create(new_contact)
            contact_id = create_result['id']

        # Update DataFrame with the Salesforce Contact ID
        register_entries.at[index, 'mbfc__Celebrant__c'] = contact_id

    except SalesforceException as e:
        print(f"Error processing row {index}: {e}")



In [ ]:
# @title Map Contact IDs to Register Entries

register_entries_2 = register_entries

register_entries_2['mbfc__Contact__c'] = register_entries['RecordNumber'].map(contact_map)


In [ ]:
# @title Append Job_Id__c
register_entries_2['Archdpdx_Job_Id__c'] = curr_job_id

## Generate an External ID

In [ ]:
def create_external_id(row):
    record_number = str(row['RecordNumber']).replace(' ', '').replace('-', '')
    entry_type = str(row['mbfc__Register_Entry_Type__c']).replace(' ', '').replace('-', '')

    # Check whether to use Type or Notation Type based on what's available
    if 'mbfc__Type__c' in row and not pd.isna(row['mbfc__Type__c']):
        type_field = str(row['mbfc__Type__c']).replace(' ', '').replace('-', '')
    elif 'mbfc__Notation_Type__c' in row and not pd.isna(row['mbfc__Notation_Type__c']):
        type_field = str(row['mbfc__Notation_Type__c']).replace(' ', '').replace('-', '') + str(row['mbfc__Ordination_Type__c']).replace(' ', '').replace('-', '')
    else:
        type_field = 'Unknown'

    return f"{record_number}_{entry_type}_{type_field}"

In [ ]:
# Assuming your DataFrame is named `register_entries`
register_entries_2['Archdpdx_Migration_ID__c'] = register_entries.apply(create_external_id, axis=1)

if register_entries['Archdpdx_Migration_ID__c'].duplicated().any():
    print("Warning: There are duplicate external IDs.")
    # Optionally, show the duplicates
    duplicates = register_entries[register_entries['external_id'].duplicated(keep=False)]
    print(duplicates)
else:
    print("All external IDs are unique.")


In [ ]:
# Drop unnecessary columns:
register_entries_2.drop(['RecordNumber', 'Prelate', 'Salutation', 'FirstName', 'MiddleName', 'LastName', 'Suffix'], axis=1, inplace=True)

In [ ]:
register_entries_staging = register_entries_2

In [ ]:
# Remove all NaN values:
register_entries_staging.fillna('', inplace=True)

# Rename columns
register_entries_staging = register_entries_staging.rename(columns={
    'Place': 'Location_text__c',
    'Date': 'mbfc__Event_Date__c'
})


In [ ]:
# @title Sent to CSV
# register_entries_staging.to_csv('register_entries_staging.csv', encoding='utf-8-sig')

In [ ]:
# @title Upsert Register Entry Records

bulk_data = []
for row in register_entries_staging.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)

# Keep the batch <100 as I've been getting an exceptionCode: 'InvalidBatch', 'exceptionMessage': 'Records not processed'
reg_entry_upsert = sf.bulk.mbfc__Sacrament__c.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
reg_entry_upsert_results = pd.DataFrame(reg_entry_upsert)

In [ ]:
# Print upsert results to local file

keys = reg_entry_upsert[0].keys()

with open('register_entry_results', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, keys)
    writer.writeheader()
    writer.writerows(reg_entry_upsert)

# Users

In [ ]:
# df_users = df_contacts[df_contacts['Access Permission'].isna() == False]
# df_users = df_users[['Record Number', 'Common Name', 'Sort Name', 'Type(s)', 'ContactRecordType', 'Login ID', 'Access Permission']]
# df_users.sort_values('Access Permission')
# df_users.to_csv(f'/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/working/users_working.csv')

# AFFILIATIONS







In [ ]:
# @title Import Assignments.csv

import pandas as pd

# Define a mapping dictionary of OriginalTable to AccRecordType name
organization_mapping = {
    'Offices': 'Organization',
    'Parishes': 'Church',
    'RelCommunities': 'Religious',
    'Schools': 'School',
    'Vicariates': 'Deanery',
    'NewmanCenters': 'Organization'
}

df_affiliations = (
    pd.read_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Assignments (1).csv')
    .set_index('Record Number', verify_integrity=True)
    .drop(index='recNum', errors='ignore')  # Added errors='ignore' to prevent errors if 'recNum' does not exist
    .drop(columns=['Historic Name'], errors='ignore')  # Added errors='ignore' for the same reason
    .rename(columns=lambda x: x.replace(' ', '_'))  # Remove whitespace in column names
    .assign(Account_Ext_Id=lambda df: df['Organization_Table_Name'].map(organization_mapping).str.capitalize() + '_' + df['Organization_Table_Link'])
    .assign(mbfc__Person__r=lambda df: df['Assigned_Person'].apply(lambda x: {'Adpdx_Migration_Id__c': x}))
    .assign(mbfc__Context__r=lambda df: df['Account_Ext_Id'].apply(lambda x: {'Adpdx_Migration_Id__c': x}))
    .assign(mbfc__Use_Custom_Title__c= True)
    # .assign(Archdpdx_Migration_Id__c= df_affiliations.index)
    .drop(columns=[
        'Assigned_Person'
        ,'Organization_Table_Name'
        ,'Organization_Table_Link'
        ,'Projected_Term_End_Date'
        ,'Term_Number'
        ,'Leave_Type' # Leave out 'Leave_Type' until mapped properly
        ])
    .rename(columns={
        'Duty_Load': 'Duty_Load__c',
        'Start_Date': 'mbfc__Start_Date__c',
        'End_Date': 'mbfc__Completion_Date__c',
        'Assignment_Title': 'mbfc__Custom_Title__c',
        'Archdiocesan_Assignment': 'ADPDX_Archdiocesan_Assignment__c',
    })
    .replace({'ADPDX_Archdiocesan_Assignment__c': {'Yes': True, 'No': ''}})
    .fillna('')
)

# Display a sample of the DataFrame to check the new structure
df_affiliations.sample(10)



In [ ]:
# @title Create ExternalID and Job Id
df_affiliations['Archdpdx_Migration_Id__c'] = df_affiliations.index

df_affiliations['Archdpdx_Job_Id__c'] = curr_job_id

df_affiliations.sample(5)

In [ ]:
df_affiliations.to_csv('affiliations_staging.csv')

In [ ]:
# @title Upsert Register Entry Records

bulk_data = []
for row in df_affiliations.itertuples(index=False):
    d = row._asdict()
    # del d['Index']
    bulk_data.append(d)



In [ ]:
# Keep the batch <100 as I've been getting an exceptionCode: 'InvalidBatch', 'exceptionMessage': 'Records not processed'
affiliation_upsert = sf.bulk.mbfc__Placement__c.upsert(data=bulk_data, external_id_field='Archdpdx_Migration_Id__c', batch_size=100, use_serial=False)
affiliation_upsert_results = pd.DataFrame(affiliation_upsert)

affiliation_upsert_results

In [ ]:
# @title Get Accounts and Contacts xref data from SF

# get SF Account
get_all_accounts = 'Select Archdpdx_Migration_Id__c, id, Name, RecordTypeId, Type, mbfc__Parish_Code__c, Job_Id__c from Account'

# get list of records, add to dataframe
sf_accounts = sf.query(get_all_accounts)
df_sf_accounts = pd.DataFrame(sf_accounts['records'])
df_sf_accounts = df_sf_accounts.drop(columns = 'attributes')
df_sf_accounts


In [ ]:
# get SF Contacts
get_all_contacts = 'Select Archdpdx_Migration_Id__c, id, Name, Archdpdx_Job_Id__c from Contact'

# get list of records, add to dataframe
sf_contacts = sf.query(get_all_contacts)
df_sf_contacts = pd.DataFrame(sf_contacts['records'])
df_sf_contacts = df_sf_contacts.drop(columns = 'attributes')
df_sf_contacts

In [ ]:
# @title Map in SF Contact and Account IDs

# Post-Migration Manual Updates

1. Convert 'Offices' that are ADPDX Pastoral Centre offices into record type: 'Groups', and set their parentID to the Diocese (there are just 6 of these accounts).
1. Update the Religous Order records 'Religious Superior' lookup.
1. Set 'organization type' field value for each account in the 'organization' load: Offices, Newman Centres, Schools, Organizations
